**Correciton setup**

In DarIA the setup of images consists of three steps:
- Color correction, 
- drift correction and,
- curvature/shape correction. 

The color correction maps the colorspace of the image to fit with the one of a classic color checker (Only usable if a color checker is present in the image). Drift correction is useful when a series of images is analyzed. Currently, it works by translating an image so that the color checkers overlap between images. Curvature/shape correction corrects for geometrically distorted features in the image by removing curvature (obtaining a 2D representation of the object of interest in the image), and cropping the image.

In this notebook, the setup of correction is demonstrated. First the color correction and drift correction is initialized on on a base image, by defining the location of the color checker. Then, a 5-step procedure for setting up curvature correction of an image is performed. Finally, an example of how load the information related to the correction procedure is performed.


First, DarIA needs to be imported.

In [ ]:
import daria as da
# Set matplotlib to plot interactive windows, requires the installation of ipympl (pip install ipympl, or conda install -c conda-forge ipympl)
%matplotlib ipympl

**Setup: Color Correction**

To set up the color correction all that is needed is to specify the pixel coordines in (x,y), i.e., (col,row) format, of the
marks on the color checker. The first coordinate is associated to the mark
closest to the brown swatch. Continue in counter-clockwise direction.
NOTE: That this example uses a crudely coarsened image. Thus, the marks
are not very obvious. They are small white L's.

A way to find the small white L's is to plot the image using matplotlib.pyplot's imshow function on the base image, zooming in on the color checker and reading the pixel coordinates of the L's.


In [ ]:
roi_color_checker = [
        [146, 196],
        [220, 199],
        [220, 95],
        [148, 91],
    ]
color_correction = da.ColorCorrection(
    roi=roi_color_checker,
)

**Setup: Drift Correction**

The drift correction requires a base image that all of the other images should be aligned to and a roi campturing the color checker. Naturally, the same roi can be used to initialize the drift correction. 

Note that both the color and drift corrections have built'in a config dictionary that can be exported to a json file. Moreover, both of the corrections can read from a config file (or directory) instead of reading the roi-list.

In [ ]:
drift_correction = da.DriftCorrection(base = "../images/baseline_with_laser_grid.jpg", roi=roi_color_checker)

**Setup: Curvature Correction**

The main idea of the curvature correction setup is to work through a reference image and determine the procedure for correcting the curvature. Then, the entire procedure (in form of different parameters) are stored in a config dictionary inside the CurvatureCorrection class, and is to be used to directly curvature correct future images with the same camera setup.

**Step 1: Initialization**

First the CurvatureCorrection object is initialized with the image_source (this can either be the path to an image or an image matrix), and the height and width of the image. 

Here, we also initialize it with the config directory of the color correction to be able to save one general config file in the end.

Note: The curvature correction object can directly be initialized from a json file with the curvature correction parameters as well, more on that later in the notebook.

In [ ]:
curv_correction = da.CurvatureCorrection(image_source = "../images/baseline_with_laser_grid.jpg", config = color_correction.config, width = 2.8, height = 1.5)

**Step 2: Pre-bulge correction**

The second step is to apply a pre-bulge correction. This is done to correct for the camera's tendency to bulge vertical lines near the edges of the image outwards. Preferably, an as small as possible parameter should be chosen below such that all straight vertical lines either becomes straight or curves slightly inwards. 

In [ ]:
#Apply setup pre-bulge correction
curv_correction.pre_bulge_correction(horizontal_bulge = 5e-9)

#Show the effect on the image
curv_correction.show_image()

**Step 3: Crop image**

Step 3 is to crop the image such that the corners of the FluidFlower appears in the corner of the image. The pixel values of the four corners of the FluidFlower should be provided, starting with the upper left corner and proceeding counter-clockwise. The pixel values can be read by mousing over desired points in the image above if it is made interactive. The points are indexed using a [col,row] rule with col increasing towards the right, and row increasing downwards.

In [ ]:
curv_correction.crop([
        [11, 8],
        [16, 1755],
        [3165, 1748],
        [3165, 5],
    ])

#In order to make interactive windows below, the ipympl options must be toggled
%matplotlib ipympl
curv_correction.show_image()

**Step 4: Bulge correction**

Step 4 is the actual bulge correction. Here, the number of pixels that the image curves inwards on each of the side should be provided (can be read from the plot above if the window is interactive).

In [ ]:
curv_correction.bulge_corection(left = 0, right = 0, top = 53, bottom = 57)

#In order to make interactive windows below, the ipympl options must be toggled
%matplotlib ipympl
curv_correction.show_image()

**Step 5: Stretch**

The final modification is to correct for potential stretch in the image. To do this we need one point that has been stretched (point_source), the point that it has been stretched from (point_destination), and the stretch center (stretch_center). The parameters can for example be found by drawing a grid on the gridded image (using daria functionality) and finding the points on the laser grid and the drawn grid. The stretch center will be where the grids align, a point_source is a laser grid line, and a point_destination is the corresponding point on the drawn grid. 

Important remark: Choose the point_destination and the point_source close to one of the four corners of the image for optimal aaccuracy.


In [ ]:
gridded_image = da.Image(curv_correction.temporary_image, width = curv_correction.width, height = curv_correction.height, color_space = "RGB")
double_gridded_image = gridded_image.add_grid(dx = 0.1, dy = 0.1, color=(100,100, 0), thickness= 5)

#In order to make interactive windows below, the ipympl options must be toggled
%matplotlib ipympl
double_gridded_image.plt_show()


In [ ]:
curv_correction.stretch_correction(point_source=[237,223], point_destination= [227, 225], stretch_center= [1476, 1020])

#Turn off interactive windows
%matplotlib inline
curv_correction.show_image()

**Applying the full procedure correction to a new image with the same setup**

Once the curvature correction has been set up properly as above, a config dictionary has automatically been created (can be reached with curv_correction.config). Applying the same settings to a new image can now be done just by initializing a DarIA image with the curvature correction object.

In [ ]:
corrected_image = da.Image("../images/co2_0.jpg", color_correction = color_correction, drift_correction = drift_correction, curvature_correction=curv_correction, width = 2.8, height = 1.5)
corrected_image.plt_show()

**Saving the config to an external json-file**

The config file can also be saved as a json file for later use, by applying the method below and providing a desired path to where the file should be stored.

In [ ]:
curv_correction.write_config_to_file("../images/config.json")

**Creating corrections from json-config file**

One could also directly initialize all of the corrections from a config file like the one we just created by using the method below.

In [ ]:
new_curv_correction = da.CurvatureCorrection(config = "../images/config.json")
new_drift_correction = da.DriftCorrection(base = "../images/baseline_with_laser_grid.jpg", config = "../images/config.json")
new_color_correction = da.ColorCorrection(config = "../images/config.json")

# The newly created curvature correction can then be applied to a new image and will work without any further setup
new_corrected_image = da.Image("../images/co2_1.jpg", color_correction = new_color_correction, drift_correction= new_drift_correction, curvature_correction = new_curv_correction, width = 2.8, height = 1.5)
new_corrected_image.plt_show()